In [ ]:
import numpy as np
from matplotlib import colors
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pickle
import signac as sg
from scipy.special import gamma
import copy as copy
import scipy
from global_functions import adjustmaps
import h5py
from scipy.interpolate import make_lsq_spline
from itertools import product
import os

# Define/load things non-specific to a given set of results
metric = 'lambda_s'
Aeff = 7.29
t_final = 300
ncell_tot = 87_993
c = 1.42
with sg.H5Store('shared_data.h5').open(mode='r') as sd:
    b_vec = np.array(sd['b_vec'])
tau_vec = b_vec * gamma(1+1/c)
tauc_methods = ["flat"]
results_pre = 'gte_thresh' 

# Update global plotting parameters
rc('axes', labelsize=21)  # Font size for x and y labels
rc('axes', titlesize=16)
rc('xtick', labelsize=19)  # Font size for x-axis tick labels
rc('ytick', labelsize=19)  # Font size for y-axis tick labels
rc('lines', markersize=15)  
rc('lines', linewidth=5.5)
rc('legend', fontsize=19)
rc('font', family='sans-serif')
rc('font', serif=['Computer Modern Sans Serif'] + plt.rcParams['font.serif'])
rc('font', weight='light')
histlw = 5.5
cbar_lpad = 30
dpi = 50
# dpi = 200

# Function to read in things specific to given results as global variables
def set_globals(results_pre):
    if metric == 'lambda_s':
        globals()['metric_lab'] = r'$S$'
        globals()['rob_metric_lab'] = r'$S^*$'
        globals()['mean_metric_lab'] = r'$\bar{\lambda}(\tau)$'
    if metric == 'P_s':
        globals()['metric_lab'] = r'$S_{meta}$'
        globals()['rob_metric_lab'] = r'$\S_{meta}^*$'
        globals()['mean_metric_lab'] = r'$<P_s>$'
    if metric == 's':
        globals()['metric_lab'] = r'$s_{meta}$'
        globals()['rob_metric_lab'] = r'$\s_{meta}^*$'
        globals()['mean_metric_lab'] = r'$<s>$'
    globals()['fn_prefix'] = f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
    #globals()['fig_prefix'] = f"{results_pre}/figs/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
    globals()['fig_prefix'] = os.path.join('/','Volumes', 'Macintosh HD', 'Users', 'patrick', 
                                           'Google Drive', 'My Drive', 'Research', 'Regan', 'Figs/')

    # Load things saved specific to these results
    globals()['metric_all'] = np.load(f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/metric_all.npy")
    globals()['tau_all'] = np.load(f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/tau_all.npy")
    globals()['C_vec'] = np.load(fn_prefix + "C_vec.npy")
    globals()['C_i_vec'] = np.arange(len(C_vec))[::2]
    globals()['ncell_vec'] = np.load(fn_prefix + "ncell_vec.npy")
    globals()['slice_left_all'] = np.load(fn_prefix + "slice_left_all.npy")
    eps_axes = {}
    with h5py.File(fn_prefix + "eps_axes.h5", "r") as handle:
        for key in handle.keys():
            eps_axes.update({key: handle[key][()]})
    globals()['eps_axes'] = eps_axes

# Read in maps and convert fdm to tau, used by multiple plots below
ul_coord = [1500, 2800]
lr_coord = [2723, 3905]
usecols = np.arange(ul_coord[0],lr_coord[0])
sdmfn = "../shared_maps/SDM_1995.asc"
sdm = np.loadtxt(sdmfn,skiprows=6+ul_coord[1],
                         max_rows=lr_coord[1], usecols=usecols)
fdmfn = '../shared_maps/FDE_current_allregions.asc'
fdm = np.loadtxt(fdmfn,skiprows=6+ul_coord[1],
                         max_rows=lr_coord[1], usecols=usecols)
sdm, fdm = adjustmaps([sdm, fdm])
delta_t = 30
b_raster = delta_t / np.power(-np.log(1-fdm), 1/c)
tau_raster = b_raster * gamma(1+1/c)
maps_filt = (sdm > 0) & (fdm > 0)
tau_flat = tau_raster[maps_filt] 
mapindices = np.argwhere(maps_filt)
tau_argsort = np.argsort(tau_flat)
tau_sorted = tau_flat[tau_argsort]

In [ ]:
set_globals(results_pre)
x_all = np.load(fn_prefix + '/x_all.npy')
meta_metric_all = np.load(fn_prefix + '/meta_metric_all.npy')
meta_metric_all = meta_metric_all[:,0]

# Create a filter for the baseline scenario
zero_eps_mask = np.all(x_all[:, 3:] == 0, axis=1)

maxrob = np.load(fn_prefix + "maxrob.npy")
argmaxrob = np.load(fn_prefix + "argmaxrob.npy")
rob_thresh_vec = np.load(fn_prefix + "rob_thresh_vec.npy")
rob_all = np.load(fn_prefix + "rob_all.npy")

### Looking at raw samples

In [ ]:
# q_vec = np.arange(0.1, 0.9, 0.1)
q_vec = np.arange(0.05, 0.85, 0.05)

# results = {}
keys = ['delta_range', 'delta_median', 'median', 'range', 'delta_n', 'delta_l', 'n', 'l',
        'delta_taul', 'delta_tauh']
results = {key: np.full((C_vec.size, q_vec.size), np.nan) for key in keys}
results['n_baseline'] = np.full(C_vec.size, np.nan)
results['l_baseline'] = np.full(C_vec.size, np.nan)

for C_i, C in enumerate(C_vec):
    for q_i, q in enumerate(q_vec):
        if q_i == 0:
            # First get optimal tau slice stats from optimal decisions under baseline
            zeroeps_filt = np.all(x_all[:, 3:] == 0, axis=1)
            _filt = zeroeps_filt & (x_all[:,0] == C)
            argmax = np.nanargmax(meta_metric_all[_filt])
            S_opt_baseline = meta_metric_all[_filt][argmax]
            n_opt_baseline, sl_opt_baseline = x_all[_filt,:][argmax][1:3].astype(int)
            tau_slice_baseline = tau_sorted[sl_opt_baseline:sl_opt_baseline+n_opt_baseline]
            range_baseline = tau_slice_baseline.max() - tau_slice_baseline.min()
            median_baseline = np.median(tau_slice_baseline)
            
            results['n_baseline'][C_i] = n_opt_baseline
            results['l_baseline'][C_i] = sl_opt_baseline

        # Now get the same statistics for (1-q) * optimal S baseline
        Sstar_rob_i = np.argmin(np.abs(rob_thresh_vec - ((1 - q) * S_opt_baseline)) )
        n_opt_rob = ncell_vec[int(argmaxrob[Sstar_rob_i, C_i][0])]
        sl_opt_rob = slice_left_all[int(argmaxrob[Sstar_rob_i, C_i][1])]
        tau_slice_rob = tau_sorted[sl_opt_rob:sl_opt_rob+n_opt_rob]
        range_rob = tau_slice_rob.max() - tau_slice_rob.min()
        median_rob = np.median(tau_slice_rob)

        results['delta_median'][C_i, q_i] = median_rob - median_baseline
        results['delta_range'][C_i, q_i] = range_rob - range_baseline
        results['median'][C_i, q_i] = median_rob
        results['range'][C_i, q_i] = range_rob
        results['delta_n'][C_i, q_i] = n_opt_rob - n_opt_baseline
        results['delta_l'][C_i, q_i] = sl_opt_rob - sl_opt_baseline
        results['n'][C_i, q_i] = n_opt_rob
        results['l'][C_i, q_i] = sl_opt_rob
        results['delta_taul'][C_i][q_i] = tau_sorted[sl_opt_rob] - tau_sorted[sl_opt_baseline]
        results['delta_tauh'][C_i][q_i] = tau_sorted[sl_opt_rob+n_opt_rob] - tau_sorted[sl_opt_baseline+n_opt_baseline]

In [ ]:
results['delta_l+delta_n'] = (results['delta_l'] + results['delta_n']) #/ np.abs(results['delta_n'])

In [ ]:
# Define reference indices for per population tau
tau_indices = np.arange(tau_sorted.size)

# Set the R value we're plotting
C = 9
assert C in C_vec/ncell_tot
C_i = (C_vec/ncell_tot == C).argmax()
# print(f"at baseline for C={C_vec[C_i]} \n optimal n,l = {results['n_baseline'][C_i], results['l_baseline'][C_i]} \n")

for q in [0.05, 0.45, 0.8]:
# for q in [0.5]:
    # Set the S^* value we're plotting
    assert np.any(np.isclose(q_vec, q))
    q_i = np.isclose(q_vec, q).argmax()

    # Get the optimal decision at this {S^*, R} combination
    n_opt = results['n'][C_i, q_i]
    l_opt = results['l'][C_i, q_i]
    # print(f'at q={q}\n optimal n,l = {n_opt, l_opt} \n')

    # Define results vector
    results_vector = np.full(tau_sorted.size, np.nan)
    # where each population is given a number to indicate optimality under:
    #   1 -> baseline condiitons only
    #   2 -> baseline and uncertain conditions (risk aversion)
    #   3 -> uncertain conditions only
    #   0 -> everything else
    
    # Create relevant masks
    baseline_mask = (tau_indices > results['l_baseline'][C_i]) & (tau_indices < results['l_baseline'][C_i] + results['n_baseline'][C_i])
    uncertain_mask = (tau_indices > l_opt) & (tau_indices < l_opt + n_opt)
    baseline_only_mask = baseline_mask & (~uncertain_mask)
    uncertain_only_mask = uncertain_mask & (~baseline_mask)
    both_mask = baseline_mask & uncertain_mask
    neither_mask = ~(baseline_mask | uncertain_mask)
    
    # Use masks to assign values to each population
    results_vector[neither_mask] = 0
    results_vector[baseline_only_mask] = 1
    results_vector[both_mask] = 2
    results_vector[uncertain_only_mask] = 3
    
    # Define colormaping for categories
    custom_colors = ['lightgrey', 'coral', 'orchid', 'blueviolet']
    labels = ['neither', 'baseline only', 'both', 'uncertain only']
    cmap = colors.ListedColormap(custom_colors)
    vmin = 0; vmax = len(custom_colors) - 1
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    
    ### TAU DISTRIBUTION VIZ ###
    
    stack_data = [tau_sorted[results_vector == i] for i in range(len(custom_colors))]
    
    bins = np.linspace(min(tau_flat), 50, 80)
    
    # Plot the stacked histogram
    fig, ax = plt.subplots(figsize=(8,5))
    ax.hist(
        stack_data,
        bins=bins,
        stacked=True,
        color=custom_colors,
        label=[labels[i] for i in range(len(custom_colors))]
    )

    ax.set_xlabel(r'$\tau$')
    ax.set_ylabel(r'baseline $\tau$ frequency')
    ax.set_title(f'$q=${q}')
    ax.legend()
    
    ### GEOGRAPHICAL MAP ###

    mapi_sorted = mapindices[tau_argsort].T

    colored_data = np.ones(maps_filt.shape + (4,)) * np.nan #colors in rgba
    colored_data[mapi_sorted[0], mapi_sorted[1]] = cmap(norm(results_vector))
    # Color background
    colored_data[maps_filt == False] = colors.to_rgba('black', alpha=0.3)
    # Crop out border where all nans
    nonzero_indices = np.nonzero(maps_filt)
    row_min, row_max = nonzero_indices[0].min(), nonzero_indices[0].max()
    col_min, col_max = nonzero_indices[1].min(), nonzero_indices[1].max()
    colored_data = colored_data[row_min:row_max + 1, col_min:col_max + 1]

    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(colored_data)#, aspect='auto')
    ax.set_yticks([])
    ax.set_xticks([])

space

In [ ]:
labels = [r'$\Delta(\tau~\text{range})$', r'$\Delta(\tau~\text{median})$', 
          r'$\tau~\text{range}$', r'$\tau~\text{median}$',
          r'$\Delta l + \Delta n$',
          r'$\Delta n$', r'$\Delta l$', r'$n$', r'$l$',
          r'$\Delta \tau_l$', r'$\Delta \tau_h$']
# cmap = copy.copy(cm.RdPu_r)
# cmap = copy.copy(cm.Wistia)
cmap = copy.copy(cm.cool)
vmin = C_vec.min() if (len(C_vec) > 1) else 0
norm = colors.Normalize(vmin=vmin/ncell_tot, vmax=C_vec.max()/ncell_tot)

# Restrict the range of plotting to a desired q value
q_lim = 0.85
q_mask = q_vec <= q_lim 

for key_i, key in enumerate(['delta_range', 'delta_median', 'range', 'median',
                             'delta_l+delta_n', 'delta_n', 'delta_l', 'n', 'l',
                             'delta_taul', 'delta_tauh']):
    fig, ax = plt.subplots(figsize=(9,5))
    
    for C_i, C in enumerate(C_vec):
        # if (C_i) % 2 == 0: continue
        # if C_i < 3: continue
        # if C_i > 3: continue
        # if C_i != len(C_vec) - 1: continue
        # if (C/ncell_tot) < 9: continue
        # if (C/ncell_tot) != 5: continue
        color = cmap(norm(C/ncell_tot))
        ax.scatter(q_vec[q_mask]*100, results[key][C_i, q_mask], marker='o', c=color, alpha=0.8)
        
    # ax.set_xlabel(r'$q$; $S^*=(1-q)\text{max}(S_{baseline})$')
    ax.set_xlabel(r'% of $\text{max}(S_{baseline})$ sacrificed')
    ax.set_ylabel(labels[key_i])
    
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(C_vec)
    cbar = fig.colorbar(sm, label=r'$R~/~n_{tot}$', ax=ax)
    
    if key not in ['range', 'median', 'n', 'l']:
        ax.axhline(0, ls='--', lw=1, c='k')
    # ax.set_xlim(0.025,0.525)

### Interpolate $\omega$ by way of $S(R, n, l, ...)$

#### 2nd attempt

In [ ]:
C = 9
assert np.any(np.isclose(C_vec/ncell_tot, C))
C_i = np.isclose(C_vec/ncell_tot, C).argmax()

# First, filter robustness values for this R value
...

# Unravel the filtered robustness values into y_obs
...

# Get the corresponding decision parameter values for each sample
...

In [ ]:
# Class to rescale inputs and outputs to [0,1] for numerical stability
# Also store descalers to interpret interpolated values
class Rescaler:
    def __init__(self, mins, maxes):
        """
        mins: vector of minima
        maxes: vector of maxima
        """
        self.mins = mins
        self.maxes = maxes

    def rescale(self, x):
        return (x - self.mins) / (self.maxes - self.mins)
    
    def descale(self, x):
        return (x * (self.maxes - self.mins)) + self.mins
    
# Rescale the y values (i.e. the robustness values)
y_rescaler = Rescaler(y_obs.min(axis=0), y_obs.max(axis=0))
y_obs = x_rescaler.rescale(y_obs) 
    
# Rescale the x values (i.e. the decision parameters)
x_rescaler = Rescaler(x_obs.min(axis=0), x_obs.max(axis=0))
x_obs = x_rescaler.rescale(x_obs) 

In [ ]:
# Now interpolate the robustness as a function of decision parameters and
# stability threshold S^*, using an interpolator that can handle noise
...

#### 1st attempt

In [ ]:
# from scipy.interpolate import RBFInterpolator
from sklearn.neighbors import KNeighborsRegressor
# import faiss

C = 9
assert np.any(np.isclose(C_vec/ncell_tot, C))
C_i = np.isclose(C_vec/ncell_tot, C).argmax()

# Filter parameter and metric metric values into xobs and yobs, respectively
indices = np.nonzero(x_all[:, 0] == (C*ncell_tot))[0]
x_obs = x_all[indices, 1:]
y_obs = meta_metric_all[indices]

# Class to rescale inputs and outputs to [0,1] for numerical stability
# Also store descalers to interpret interpolated values
class Rescaler:
    def __init__(self, mins, maxes):
        """
        mins: vector of minima
        maxes: vector of maxima
        """
        self.mins = mins
        self.maxes = maxes

    def rescale(self, x):
        return (x - self.mins) / (self.maxes - self.mins)
    
    def descale(self, x):
        return (x * (self.maxes - self.mins)) + self.mins
    
x_rescaler = Rescaler(x_obs.min(axis=0), x_obs.max(axis=0))
x_obs = x_rescaler.rescale(x_obs)

y_rescaler = Rescaler(y_obs.min(), y_obs.max())
y_obs = y_rescaler.rescale(y_obs)

In [ ]:
# Create the interpolator
# interp = RBFInterpolator(x_obs, y_obs, neighbors=50)

interp = KNeighborsRegressor(n_neighbors=100, weights='distance', algorithm='auto', n_jobs=6)
interp.fit(x_obs, y_obs)

# d = x_obs.shape[1]
# index = faiss.IndexFlatL2(d)
# index.add(x_obs.astype('float32'))

In [ ]:
# Create a grid of uncertain param samples to evaluate robustness with
uncertain_params = ['mu_tau', 'sigm_tau', 'mu_tauc', 'sigm_tauc', 'demographic_index']
num_points = 6
linspaces = [np.linspace(0, 1, num_points) for _ in range(5)] # Still in rescaled space so all [0,1]
grid_arrays = np.meshgrid(*linspaces)
uncertainty_samples = np.vstack([array.ravel() for array in grid_arrays]).T

# Initialize set of full samples for which the decision parameters will be filled in
empty_samples = np.full((len(uncertainty_samples), 2), np.nan)
full_samples = np.column_stack((empty_samples, uncertainty_samples))

In [ ]:
# Evaluate at an example point
n = 0.5
l = 0.2

full_samples[:, 0] = n
full_samples[:, 1] = l

# interp(full_samples)

interp.predict(full_samples)

# k = 50  # Number of neighbors
# distances, indices = index.search(full_samples.astype('float32'), k)
# Y_neighbors = y_obs[indices]  # shape: (n_query, k)
# y_pred = Y_neighbors.mean(axis=1)  # shape: (n_query,)

In [ ]:
# Optimize at a particular S^* value
Sstar = 0.5


##### RBF example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RBFInterpolator
from scipy.stats.qmc import Halton

rng = np.random.default_rng()
xobs = 2*Halton(2, seed=rng).random(100) - 1
yobs = np.sum(xobs, axis=1)*np.exp(-6*np.sum(xobs**2, axis=1))

xgrid = np.mgrid[-1:1:50j, -1:1:50j]
xflat = xgrid.reshape(2, -1).T
yflat = RBFInterpolator(xobs, yobs)(xflat)
ygrid = yflat.reshape(50, 50)

fig, ax = plt.subplots()
ax.pcolormesh(*xgrid, ygrid, vmin=-0.25, vmax=0.25, shading='gouraud')
p = ax.scatter(*xobs.T, c=yobs, s=50, ec='k', vmin=-0.25, vmax=0.25)
fig.colorbar(p)
plt.show()